In [1]:
# # To support both python 2 and python 3# To su 
# from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Nauczmy RNN, który będzie klasyfikował obrazy MNIST.
* Conwolucyjna sieć neuronowa (CNN) lepiej pasuje do klasyfikacji obrazów, ale taki przykład pokaże nam jak taką siec można użyć w praktyce.
* Każdy obraz potraktujemy jako ciąg 28 wierszy po 28 pikseli każdy (ponieważ każdy obraz MNIST to 28 × 28 pikseli).
* Użyjemy warst składających się 150 ewkurencyjnych neuronów oraz warstwy fully connected zawierającej 10 neuronów wyjściowych (jeden na każdą klasę) połączonych z wyjściem z ostatniego kroku czasowego, a następnie warstwę softmax layer.

  * Faza budowy jest dość prosta i jest prawie taka sam jak w klasyfikator MNIST (CNN)
  * Z tą różnicą, że RNN zastępuje ukryte warstwy. 
  * Należy zauważyć, że waretwa fully connected jest połączona z tensorem stanów, który zawiera tylko stan końcowy RNN (tj. 28 wyjście). 
  * Zauważ także, że y jest symbolem zastępczym dla klas docelowych.

In [3]:
tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.92 Test accuracy: 0.9128
1 Train accuracy: 0.97333336 Test accuracy: 0.9382
2 Train accuracy: 0.98 Test accuracy: 0.9574
3 Train accuracy: 0.9533333 Test accuracy: 0.9532
4 Train accuracy: 0.99333334 Test accuracy: 0.9607
5 Train accuracy: 0.9866667 Test accuracy: 0.9688
6 Train accuracy: 0.9866667 Test accuracy: 0.9642
7 Train accuracy: 0.97333336 Test accuracy: 0.9695
8 Train accuracy: 0.9533333 Test accuracy: 0.9709
9 Train accuracy: 0.9866667 Test accuracy: 0.9638
10 Train accuracy: 0.98 Test accuracy: 0.9741
11 Train accuracy: 0.9866667 Test accuracy: 0.9714
12 Train accuracy: 0.99333334 Test accuracy: 0.9746
13 Train accuracy: 0.97333336 Test accuracy: 0.9732
14 Train accuracy: 1.0 Test accuracy: 0.9798
15 Train accuracy: 0.99333334 Test accuracy: 0.9765
16 Train accuracy: 0.9866667 Test accuracy: 0.9711
17 Train accuracy: 0.99333334 Test accuracy: 0.9776
18 Train accuracy: 0.9866667 Test accuracy: 0.9776
19 Train accuracy: 0.9866667 Test accuracy: 0.9753
20 T

In [7]:
X_batch.shape

(150, 28, 28)

In [10]:
X_batch[0,:,:].shape

(28, 28)